In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.distance import geodesic

In [53]:
# Read the data from the csv file
airport_info = pd.read_csv('Assignment_Data_P1/Group_4_Airport_info.csv')

# We make a dictionary with the airport code as the key the rest of the columns as sub-dictionaries
airport_dict = {}

for index, row in airport_info.iterrows():
    airport_dict[row['ICAO Code']] = row.to_dict()
    
airport_dict

{'LIRF': {'City Name': 'Rome',
  'ICAO Code': 'LIRF',
  'Latitude (deg)': 41.77354,
  'Longitude (deg)': 12.239712,
  'Runway (m)': 3900.0,
  'Population': 5671181,
  'GDP': 200.84},
 'EDDT': {'City Name': 'Berlin',
  'ICAO Code': 'EDDT',
  'Latitude (deg)': 52.5597,
  'Longitude (deg)': 13.2877,
  'Runway (m)': 2900.0,
  'Population': 5342958,
  'GDP': 234.265},
 'RJAA': {'City Name': 'Tokyo',
  'ICAO Code': 'RJAA',
  'Latitude (deg)': 35.7702,
  'Longitude (deg)': 140.3843,
  'Runway (m)': 4000.0,
  'Population': 40700000,
  'GDP': 2080.758},
 'EDDM': {'City Name': 'Munich',
  'ICAO Code': 'EDDM',
  'Latitude (deg)': 48.3538,
  'Longitude (deg)': 11.7861,
  'Runway (m)': 2800.0,
  'Population': 2926802,
  'GDP': 228.897},
 'LFPG': {'City Name': 'Paris',
  'ICAO Code': 'LFPG',
  'Latitude (deg)': 49.0128,
  'Longitude (deg)': 2.55,
  'Runway (m)': 3100.0,
  'Population': 14800000,
  'GDP': 1013.337},
 'SAEZ': {'City Name': 'Buenos Aires',
  'ICAO Code': 'SAEZ',
  'Latitude (deg)': -34

In [54]:
# Create distance matrix
# Create a list of all the airports
airport_list = airport_info['ICAO Code'].tolist()

# Create a dictionary of all the airports and their coordinates
airport_coords = {}

for i in airport_list:
    airport_coords[i] = (airport_info[airport_info['ICAO Code'] == i]['Latitude (deg)'].values[0],
                       airport_info[airport_info['ICAO Code'] == i]['Longitude (deg)'].values[0])

# Create a distance matrix
distance_dict = {}

# Fill the distance matrix in a dict with the distances between the airports
for i in airport_list:
    distance_dict[i] = {}
    for j in airport_list:
        distance_dict[i][j] = geodesic(airport_coords[i], airport_coords[j]).km

distance_dict

{'LIRF': {'LIRF': 0.0,
  'EDDT': 1201.733756862431,
  'RJAA': 9930.539562786149,
  'EDDM': 732.152536464005,
  'LFPG': 1104.1802466608858,
  'SAEZ': 11132.631769402296,
  'CYYZ': 7104.777847088536,
  'MMMX': 10240.215415047822,
  'CYVR': 9026.595945099885,
  'EDDF': 960.8878299291965,
  'KJFK': 6884.59969496344,
  'SBGL': 9149.879934580182,
  'LEMD': 1333.0021285765708,
  'ZSPD': 9191.891122558394,
  'OMDB': 4348.451296083171,
  'LTFM': 1376.6402451803178,
  'ZBAA': 8170.998825130269,
  'VIDP': 5945.306647569454,
  'EFRO': 2882.8621802599173,
  'EHEH': 1195.9914533507306,
  'EFHK': 2238.466007915691,
  'LDZA': 537.730449205408,
  'EBBR': 1175.6122782014334,
  'EIDW': 1890.7139987581077,
  'LGAV': 1087.5645035695106,
  'BIKF': 3308.9116025207964,
  'EETN': 2146.5365994067547,
  'LKPR': 938.7949518360457,
  'FACT': 8412.19511819193,
  'ESSA': 2027.0526151677066,
  'EGPH': 1939.0336821795868,
  'LFQQ': 1203.5529891309463,
  'LPPT': 1843.04067343599},
 'EDDT': {'LIRF': 1201.733756862431,
 

In [55]:
# Import aircraft info
aircraft_info = pd.read_csv('Assignment_Data_P1/Aircraft_info.csv')

print(aircraft_info.info())
aircraft_types = aircraft_info['AC_type'].tolist()

# Add a column for BT (Block Time) to the aircraft_info dataframe
aircraft_info['BT'] = 10  # [hours]
aircraft_info['AC'] = 15  # [units]

# Create a dictionary with the aircraft types as keys and the rest of the columns as sub-dictionaries
aircraft_dict = {}
for index, row in aircraft_info.iterrows():
    aircraft_dict[row['AC_type']] = row.to_dict()

aircraft_dict

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   AC_type      3 non-null      object 
 1   Speed        3 non-null      int64  
 2   Seats        3 non-null      int64  
 3   TAT          3 non-null      int64  
 4   Range        3 non-null      int64  
 5   Runway       3 non-null      int64  
 6   Lease_c      3 non-null      int64  
 7   Operating_c  3 non-null      int64  
 8   Time_c       3 non-null      int64  
 9   Fuel_c       3 non-null      float64
dtypes: float64(1), int64(8), object(1)
memory usage: 368.0+ bytes
None


{'Regional Jet': {'AC_type': 'Regional Jet',
  'Speed': 820,
  'Seats': 70,
  'TAT': 35,
  'Range': 3300,
  'Runway': 1600,
  'Lease_c': 34000,
  'Operating_c': 600,
  'Time_c': 775,
  'Fuel_c': 2.0,
  'BT': 10,
  'AC': 15},
 'Single Aisle Twin Engine Jet': {'AC_type': 'Single Aisle Twin Engine Jet',
  'Speed': 850,
  'Seats': 150,
  'TAT': 45,
  'Range': 6300,
  'Runway': 1800,
  'Lease_c': 70000,
  'Operating_c': 1250,
  'Time_c': 1400,
  'Fuel_c': 3.75,
  'BT': 10,
  'AC': 15},
 'Twin aisle twin engine jet': {'AC_type': 'Twin aisle twin engine jet',
  'Speed': 870,
  'Seats': 320,
  'TAT': 60,
  'Range': 12000,
  'Runway': 2600,
  'Lease_c': 180000,
  'Operating_c': 2000,
  'Time_c': 2800,
  'Fuel_c': 7.0,
  'BT': 10,
  'AC': 15}}

In [56]:
# Import demand info
demand_info = pd.read_csv('Assignment_Data_P1/Group_4_Demand.csv')
demand_info.rename(columns={'Unnamed: 0': 'Origin'}, inplace=True)

# Create a dictionary with the demand info
demand_dict = {}
for index, row in demand_info.iterrows():
    demand_dict[row['Origin']] = row.to_dict()

# Remove the origin column from the demand_dict
for i in demand_dict:
    demand_dict[i].pop('Origin', None)

demand_dict

{'EHEH': {'EHEH': 0,
  'EBBR': 37,
  'VIDP': 32,
  'SAEZ': 18,
  'BIKF': 4,
  'EDDT': 30,
  'MMMX': 22,
  'LFQQ': 22,
  'LGAV': 16,
  'EDDF': 26,
  'FACT': 9,
  'KJFK': 32,
  'RJAA': 46,
  'OMDB': 13,
  'LFPG': 57,
  'ZSPD': 28,
  'ZBAA': 30,
  'EFHK': 14,
  'EDDM': 24,
  'LKPR': 16,
  'CYVR': 12,
  'LTFM': 38,
  'EGPH': 8,
  'EFRO': 3,
  'CYYZ': 18,
  'LPPT': 15,
  'EIDW': 20,
  'LEMD': 29,
  'LIRF': 23,
  'SBGL': 22,
  'ESSA': 16,
  'EETN': 6,
  'LDZA': 10},
 'EBBR': {'EHEH': 33,
  'EBBR': 0,
  'VIDP': 84,
  'SAEZ': 47,
  'BIKF': 13,
  'EDDT': 70,
  'MMMX': 71,
  'LFQQ': 68,
  'LGAV': 46,
  'EDDF': 73,
  'FACT': 25,
  'KJFK': 96,
  'RJAA': 107,
  'OMDB': 41,
  'LFPG': 181,
  'ZSPD': 88,
  'ZBAA': 86,
  'EFHK': 41,
  'EDDM': 66,
  'LKPR': 46,
  'CYVR': 26,
  'LTFM': 98,
  'EGPH': 25,
  'EFRO': 8,
  'CYYZ': 58,
  'LPPT': 39,
  'EIDW': 64,
  'LEMD': 69,
  'LIRF': 62,
  'SBGL': 48,
  'ESSA': 52,
  'EETN': 20,
  'LDZA': 26},
 'VIDP': {'EHEH': 29,
  'EBBR': 78,
  'VIDP': 0,
  'SAEZ': 114,


In [57]:
# Make LIRF the hub with a dictionary of all the airports with a 1 if it is the hub and 0 if it is not
hub = "LIRF"

g = {}
for i in airport_list:
    if i == hub:
        g[i] = 1
    else:
        g[i] = 0

In [58]:
# Revenue parameters

# Load factor
LF = 0.8

# Create Yield matrix dict from formula in RPK using distance per aircraft type using the aircraft type as the key and the rest of the columns as sub-dictionaries
# Revenue generated by passengers connecting at the hub will be 10% lower 
# Formula: Yield = 5.9 ∙ dij^(−0.76) + 0.043

yield_matrix = {}
for k in aircraft_types:
    yield_matrix[k] = {}
    for i in airport_list:
        yield_matrix[k][i] = {}
        for j in airport_list:
            if i == j:
                yield_matrix[k][i][j] = 0
            else:
                yield_matrix[k][i][j] = 5.9 * (distance_dict[i][j] ** (-0.76)) + 0.043

yield_matrix

{'Regional Jet': {'LIRF': {'LIRF': 0,
   'EDDT': 0.06992760533821954,
   'RJAA': 0.048409444242827726,
   'EDDM': 0.08224233288312846,
   'LFPG': 0.07171716951350784,
   'SAEZ': 0.0479594972066314,
   'CYYZ': 0.049977078823230145,
   'MMMX': 0.048284660608056,
   'CYVR': 0.048816393354816426,
   'EDDF': 0.07491690456679825,
   'KJFK': 0.050146019628108274,
   'SBGL': 0.04875673597061972,
   'LEMD': 0.06788745977129522,
   'ZSPD': 0.04873672866660819,
   'OMDB': 0.05313252555828846,
   'LTFM': 0.06728557892472306,
   'ZBAA': 0.04927368774495451,
   'VIDP': 0.05098876368167407,
   'EFRO': 0.056847961972560174,
   'EHEH': 0.07002580717154067,
   'EFHK': 0.059783774223934535,
   'LDZA': 0.0926161307536247,
   'EBBR': 0.07038112485453549,
   'EIDW': 0.062081669675308995,
   'LGAV': 0.07205000376849223,
   'BIKF': 0.055470713046114305,
   'EETN': 0.0603273004231626,
   'LKPR': 0.07548614997834914,
   'FACT': 0.04913650249565579,
   'ESSA': 0.06109816716248087,
   'EGPH': 0.06171919391709879,

In [59]:
# Cost parameters

# All aircraft are leased, and therefore a leasing cost needs to be accounted for. 
# The weekly leasing cost is a fixed amount depending on the type of aircraft

# Fuel cost formula
# CF_kij = CF_k ∙ f ∙ dij / 1.5
# Where 
# CF_kij = fuel cost for aircraft type k on route i-j [€]
# CF_k = fuel cost for aircraft type k [galon/km]
f  = 1.42 # fuel cost [€/galon]
# dij = distance between airport i and j [km]

# Time-based costs formula
# CT_kij = CT_k ∙ dij / V_k 
# Where
# CT_kij = total time-based cost for aircraft type k on route i-j [€]
# CT_k = total time-based cost for aircraft type k [€/h]
# dij = distance between airport i and j [km]
# V_k = cruise speed for aircraft type k [km/h]

# Variable costs formula
# Op_Cost_kij = CX_kij + CF_kij + CT_kij


# Fixed leg costs
# CX_k depends on the aircraft type and is a fixed cost per flight

# Create a dictionary with the Op_Cost for each aircraft type and airport using the aircraft type as the key and the rest of the columns as sub-dictionaries

Op_Cost_dict = {}
for k in aircraft_types:
    Op_Cost_dict[k] = {}
    for i in airport_list:
        Op_Cost_dict[k][i] = {}
        for j in airport_list:
            Op_Cost_dict[k][i][j] = aircraft_dict[k]['Operating_c'] + \
                                 aircraft_dict[k]['Fuel_c'] * f * distance_dict[i][j] / 1.5 + \
                                 aircraft_dict[k]['Time_c'] * distance_dict[i][j] / aircraft_dict[k]['Speed']
            # It should be noted that for flights departing or arriving at the hub airport the operating costs can be assumed to be 30% lower due to economies of scale
            if i == hub or j == hub:
                Op_Cost_dict[k][i][j] = Op_Cost_dict[k][i][j] * 0.7

Op_Cost_dict

{'Regional Jet': {'LIRF': {'LIRF': 420.0,
   'EDDT': 2807.747272954198,
   'RJAA': 20151.17475031601,
   'EDDM': 1874.7275653575173,
   'LFPG': 2613.916379363419,
   'SAEZ': 22539.634233788587,
   'CYYZ': 14536.615957949709,
   'MMMX': 20766.475491861398,
   'CYVR': 18355.112273324437,
   'EDDF': 2329.2059971075305,
   'KJFK': 14099.139870339917,
   'SBGL': 18600.067537333212,
   'LEMD': 3068.566855324851,
   'ZSPD': 18683.54035230218,
   'OMDB': 9060.019192691561,
   'LTFM': 3155.2722452021385,
   'ZBAA': 16655.110356686273,
   'VIDP': 12232.840950456475,
   'EFRO': 6148.0127731374105,
   'EHEH': 2796.3377827303934,
   'EFHK': 4867.649968622143,
   'LDZA': 1488.426684648446,
   'EBBR': 2755.846018552247,
   'EIDW': 4176.694998947095,
   'LGAV': 2580.902248714278,
   'BIKF': 6994.538337005364,
   'EETN': 4684.993707850538,
   'LKPR': 2285.3092445053903,
   'FACT': 17134.34778154507,
   'ESSA': 4447.588745312609,
   'EGPH': 4272.702281476027,
   'LFQQ': 2811.3619395666756,
   'LPPT': 40

In [66]:
Op_Cost_dict['Regional Jet']['LIRF']['LIRF']

420.0

In [60]:
import gurobipy as gp
from gurobipy import GRB

In [61]:
# Create model

m = gp.Model("Aircraft and network model")

In [64]:
# Create variables

# w_ij: flow from airport i to airport j that transfers at the hub
w = m.addVars(airport_list, airport_list, vtype=GRB.INTEGER, name="w") 

# x_ij: direct from airport i to airport j that does not transfer at the hub
x = m.addVars(airport_list, airport_list, vtype=GRB.INTEGER, name="x")

# z_kij: number of flights from airport i to airport j with aircraft k
z = m.addVars(aircraft_types, airport_list, airport_list, vtype=GRB.INTEGER, name="z")

# # y_k: number of aircraft of type k
# y = m.addVars(aircraft_types, vtype=GRB.INTEGER, name="y")
    

In [68]:
# Create objective function to maximize profit

# Revenue
# Rev1: direct revenue from direct flights (not transferring at the hub)
rev_1 = gp.quicksum((yield_matrix[k][i][j] * x[i, j]) for k in aircraft_types for i in airport_list for j in airport_list)

# Rev2: revenue from connecting passengers
# Revenue generated by passengers connecting at the hub will be 10% lower
rev_2 = 0.9 * gp.quicksum((yield_matrix[k][i][j] * w[i, j]) for k in aircraft_types for i in airport_list for j in airport_list)

# Cost
# Cost1: Fixed weekly leasing cost for all aircraft types
cost_1 = gp.quicksum((aircraft_dict[k]['Lease_c'] * aircraft_dict[k]['AC']) for k in aircraft_types)

# Cost2: Operational costs per flight from i to j
# Op_Cost_kij = CF_kij + CT_kij + CX_k (for all aircraft types k)

cost_2 = gp.quicksum((Op_Cost_dict[k][i][j] * z[k, i, j]) for k in aircraft_types for i in airport_list for j in airport_list)

# # Cost3: Operational costs for direct flights departing or arriving at the hub
# # It should be noted that for flights departing or arriving at your hub airport the operating costs can be assumed to be 30% lower due to economies of scale
# # C_kij = CF_kij + CT_kij + CX_k (for all aircraft types k where i or j is the hub)

# cost_3 = 0.7 * (
#          gp.quicksum((aircraft_dict[k]['Fuel_c'] * (f * distance_dict[i][j] / 1.5) * z[k, i, j] ) for k in aircraft_types for i in airport_list for j in airport_list if i == hub or j == hub) + \
#          gp.quicksum((aircraft_dict[k]['Time_c'] * (distance_dict[i][j] / aircraft_dict[k]['Speed']) * z[k, i, j]) for k in aircraft_types for i in airport_list for j in airport_list if i == hub or j == hub) + \
#          gp.quicksum((aircraft_dict[k]['Operating_c'] * z[k, i, j]) for k in aircraft_types for i in airport_list for j in airport_list if i == hub or j == hub)
#          )

m.setObjective(rev_1 + rev_2 - cost_1 - cost_2, GRB.MAXIMIZE)
m.update()

In [69]:
# Special hub conditions
# TAT for flights to the hub are 50% longer than the normal TAT for each aircraft type

# Add TAT to hub to the aircraft dictionary

for i in aircraft_types:
    aircraft_dict[i]['TAT_hub'] = aircraft_dict[i]['TAT'] * 0.5


In [70]:
# Add constraints

# Constraint 1: number of passengers from airport i to airport j
# x_ij + w_ij <= demand_ij (for all i and j)
m.addConstrs((x[i, j] + w[i, j] <= demand_dict[i][j] for i in airport_list for j in airport_list), name="c1")

# Constraint 2: Transfer passengers are only if the hub is not the origin or destination
# w_ij <= demand_ij * g_i * g_j (for all i and j)
m.addConstrs((w[i, j] <= demand_dict[i][j] * g[i] * g[j] for i in airport_list for j in airport_list), "c2")

# Constraint 3: capacity verification for each flight leg
# x_ij + sum(w_im * (1 - g_j) for all m) + sum(w_mj * (1 - g_i) for all m) <= sum(z_kij * s_k * LF for all k) (for all i and j)
m.addConstrs((x[i, j] + 
              gp.quicksum(w[i, m] * (1 - g[j]) for m in airport_list) + 
              gp.quicksum(w[m, j] * (1 - g[i]) for m in airport_list) <= 
              gp.quicksum(z[k, i, j] * aircraft_dict[k]['Seats'] * LF for k in aircraft_types) 
              for i in airport_list for j in airport_list), "c3")

# Constraint 4: same departing and arriving aircrafts per airport
# sum(z_kij) = sum(z_kji) (for all i and j)
m.addConstrs((gp.quicksum(z[k, i, j] for k in aircraft_types) == gp.quicksum(z[k, j, i] for k in aircraft_types) for i in airport_list for j in airport_list), "c4")


# Constraint 5: block time verification for each aircraft total
# we should add a TAT for only incoming to the hub of 50% of the normal TAT
# sum((dij / sp_k + TAT_k + (TAT_hub for all j != hub)) * z_kij for all i and j) <= BT_k * y_k (for all k)
m.addConstrs((gp.quicksum((distance_dict[i][j] / aircraft_dict[k]['Speed'] + aircraft_dict[k]['TAT'] / 60 ) *
                          #(aircraft_dict[k]['TAT_hub'] * (1 - g[j]) / 60)) * 
                           z[k, i, j] for i in airport_list for j in airport_list) <= 
                           aircraft_dict[k]['BT'] * aircraft_dict[k]['AC'] * 7 for k in aircraft_types), "c5")


# Define matrix a[k, i, j] based on aircraft range
a = {}
for k in aircraft_types:
    for i in airport_list:
        for j in airport_list:
            a[k, i, j] = 100000 if distance_dict[i][j] <= aircraft_dict[k]['Range'] else 0

# Constraint 6: aircraft range verification
# z_kij <= a_kij (for all k, i, j)
m.addConstrs((z[k, i, j] <= a[k, i, j] for k in aircraft_types for i in airport_list for j in airport_list), "c6")

# Constraint 7: no self flights
# z_kii = 0 (for all k and i)
m.addConstrs((z[k, i, j] == 0 for k in aircraft_types for i in airport_list for j in airport_list if i == j), "c8")


# Update model
m.update()

In [71]:
# Optimize model
m.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 15450 rows, 10890 columns and 178528 nonzeros
Model fingerprint: 0x148195bc
Variable types: 0 continuous, 10890 integer (0 binary)
Coefficient statistics:
  Matrix range     [6e-01, 3e+02]
  Objective range  [1e-01, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 1e+05]
Found heuristic solution: objective -4260000.000
Presolve removed 14607 rows and 8786 columns
Presolve time: 0.07s
Presolved: 843 rows, 2104 columns, 5192 nonzeros
Variable types: 0 continuous, 2104 integer (0 binary)

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.12 seconds (0.05 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: -4.26e+06 
No other solutions better than -4.26e+06

Optimal so

In [73]:
# Print the solution as a pd.DataFrame for each aircraft type

solution_z = pd.DataFrame(columns=airport_list, index=airport_list)


for k in aircraft_types:
    for v in m.getVars():
        if v.varName[0] == 'z' and v.varName[2:6] == k:
            solution_z.loc[v.varName[7:11], v.varName[12:16]] = v.x
    print('Total flights for aircraft type %s \n' % k, solution_z, '\n')

# Print variables z values
for v in m.getVars():
    if v.varName[0] == 'z':
        print('%s %g' % (v.varName, v.x))

Total flights for aircraft type Regional Jet 
      LIRF EDDT RJAA EDDM LFPG SAEZ CYYZ MMMX CYVR EDDF  ... EIDW LGAV BIKF  \
LIRF  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
EDDT  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
RJAA  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
EDDM  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
LFPG  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
SAEZ  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
CYYZ  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
MMMX  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
CYVR  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
EDDF  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
KJFK  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
SBGL  NaN  NaN  N